In [2]:
import pandas as pd
from tqdm import tqdm
from binance.client import Client

client = Client()

### All tradable assets in this dictionnary

In [3]:
info = client.get_exchange_info()

In [9]:
symbols = [x['symbol'] for x in info['symbols']]

In [16]:
len(symbols)

2766

In [15]:
#exclude leveraged tokens
exclude = ['UP', 'DOWN', 'BEAR', 'BULL']
non_lev = [ symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]

In [14]:
len(non_lev)

2697

### Keep only usdt pair

In [17]:
relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')]

In [19]:
len(relevant)

471

In [20]:
klines = {}

### Get 1 minute candles for each symbol

In [21]:
for symbol in tqdm(relevant) : 
    klines[symbol] = client.get_historical_klines(symbol, '1m', '1 hour ago UTC')

100%|██████████| 471/471 [04:22<00:00,  1.80it/s]


In [25]:
(pd.DataFrame(klines['BTCUSDT'])[4].astype(float).pct_change() + 1 ).prod() -1 #4 column is the close

0.00021125056342108373

In [26]:
returns, symbols = [], []

for symbol in relevant:
    if len(klines[symbol]) > 0 : 
        cumret = (pd.DataFrame(klines[symbol])[4].astype(float).pct_change() + 1 ).prod() - 1
        returns.append(cumret)
        symbols.append(symbol)

In [27]:
retdf = pd.DataFrame(returns, index=symbols, columns=['ret'])

In [28]:
retdf.ret.nlargest(10)

API3USDT     0.043275
OAXUSDT      0.034763
ZECUSDT      0.024750
DYDXUSDT     0.016505
MOVRUSDT     0.015326
FTTUSDT      0.014781
RAYUSDT      0.012847
BTTCUSDT     0.011628
MAGICUSDT    0.010071
AGLDUSDT     0.008056
Name: ret, dtype: float64